### Imports

In [1]:
import pandas as pd
import numpy as np
from utility_functions import load_file, pickle_file, starting_run, finished_run
from analysis_variables import de_col_keys, de_col_values
from scipy.stats import f_oneway, sem, norm

full_dataset = load_file("summary_costs_enhanced.pickle")
filtered_dataset_codes = load_file("fully_filtered_codes.pickle")
code_category_dict = load_file("code-category-dict.pickle")

### Demographic Column Classification

In [2]:
full_dataset["Unmarried"] = full_dataset["marital_status"].ne('Married')
full_dataset["Married"] = full_dataset["marital_status"].eq('Married')
full_dataset["Uninsured"] = full_dataset["payer"].isin(['No charge', 'Self-pay'])
full_dataset["Medicare"] = full_dataset["payer"].eq('Medicare')
full_dataset["Medicaid"] = full_dataset["payer"].eq('Medicaid')
full_dataset["Private Insurance"] = full_dataset["payer"].eq('Private insurance')
full_dataset["White"] = full_dataset["race"].eq("White")
full_dataset["African American or Hispanic"] = full_dataset["race"].isin(['African-American', "Hispanic"])
full_dataset["Female"] = full_dataset["gender"].eq("Female")
full_dataset["Died"] = full_dataset["Died"].eq(1)
del full_dataset["Admitted"] #step needed in this analysis due to "Admitted" column in de_col_values
for key in de_col_keys:
    full_dataset = full_dataset.join([full_dataset[key].eq(val).rename(val).loc[full_dataset.index] for val in de_col_values[key]])

dem_dataset = full_dataset[[
    'median_zip_income', 
    'age',
    'Cost',
    "Unmarried", 
    "Married", 
    "Uninsured", 
    "Medicare",
    "Medicaid",
    "Private Insurance",
    "White",
    "African American or Hispanic",
    "Female",
    'ED Readmissions',
    'Inpatient Readmissions',
    'Died',
    *pd.core.common.flatten(de_col_values.values())
]].copy()

category_dict = {
    'Cost': 'Outcome',
    'ED Readmissions': 'Outcome',
    'Inpatient Readmissions': 'Outcome',
    'Died': 'Outcome',
    'admitted': 'Clinical Pathway',
    'alcohol-related disorders': 'Comorbidity',
    'aspirin': 'Comorbidity',
    'biliary colic with inflammation': 'Comorbidity',
    'diabetes mellitus': 'Comorbidity',
    'hyperlipidemia': 'Comorbidity',
    'ischemic heart disease': 'Comorbidity',
    'mood disorders': 'Comorbidity',
    'nicotine dependence': 'Comorbidity',
    'obesity': 'Comorbidity',
    'systemic hypertension': 'Comorbidity',
    'Medicaid': 'Insurance Status',
    'Medicare': 'Insurance Status',
    'Private Insurance': 'Insurance Status',
    'Uninsured': 'Insurance Status',
    'age': 'Demographic',
    'African American or Hispanic': 'Demographic',
    'Female': 'Demographic',
    'Married': 'Demographic',
    'Unmarried': 'Demographic',
    'White': 'Demographic',
    'median_zip_income': 'Demographic',
    **{value: 'Clinical Pathway' for value in pd.core.common.flatten(de_col_values.values())}
}

### ICD-10 Diagnosis Code Classification

In [3]:
category_list = []
visit_codes = filtered_dataset_codes.reset_index().\
                groupby("visit_link")["codes"].unique().apply(lambda x: [st.strip() for st in x])
for key, values in code_category_dict.items():
    category_list.append(visit_codes.transform(
        lambda x: any([any([code.startswith(value) for value in values]) for code in x])
    ).rename(key))
category_status = pd.DataFrame(category_list).astype("int").T
category_status.index = category_status.index.astype("int")
category_status = category_status.loc[dem_dataset.index, :] # to remove rows dropped in 2.4
pickle_file("category_status_filtered.pickle", category_status)

### Create Summary Table

In [22]:
def create_summary(groupby_col, filter_criteria="Cost >= 0"): #cost should always be positive, making this a universal filter
    num_full_dataset = dem_dataset.query(filter_criteria).join(category_status, how="inner")\
        .astype("int").join(full_dataset[groupby_col]).groupby(groupby_col)
    #         .join(full_dataset['Admitted'].map({True: 'Admitted', False: 'Not Admitted'}))\
    agg_table = num_full_dataset.agg(['mean', 'sum', 'sem']).set_index(
        num_full_dataset.count().iloc[:, 0].reset_index().apply(
            lambda row: f"{row[0]} (N={row[1]})",
            axis=1
        )
    ).T.sort_index()
    summary_table = agg_table.reset_index().set_index("level_1").groupby("level_0").aggregate(
       lambda col: (col.loc["mean"], col.loc["sum"], norm.interval(alpha=0.95,loc=0,scale=col.loc["sem"])[1]) if col.loc["mean"] != 0 else (0, col.loc["sum"], 0)
    ).applymap(
        lambda val: f"{round(val[1], 2)} ({round(val[0]*100,0)}%)" if val[0] <= 1 else f"{round(val[0], 2)} ± {round(val[2], 2)}"
    )
    summary_table["ANOVA P"] = ["p < 0.01" if f_oneway(*[x for _, x in num_full_dataset[col]]).pvalue < 0.01 else "p > 0.01" for col in agg_table.index.levels[0]]
    summary_table["category"] = summary_table.index.map(category_dict)
    summary_table.index = summary_table.index.astype("str") # just in case one of the de_col_values is a number or boolean
    summary_table = summary_table.reset_index().set_index(['category', 'level_0', 'ANOVA P']).sort_index()
    return summary_table

In [23]:
# create_summary(de_col_keys[0]).to_excel(f"../tables/Table 1 {de_col_keys[0]}.xlsx")
create_summary(de_col_keys[0])

C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


+ Suboxone (N=21)  \
category         level_0                      ANOVA P                      
Clinical Pathway + Suboxone                   p > 0.01     21.0 (100.0%)   
                 - Suboxone                   p > 0.01          0.0 (0%)   
                 0                            p > 0.01          0.0 (0%)   
                 12                           p > 0.01       5.0 (24.0%)   
                 2                            p > 0.01        1.0 (5.0%)   
                 4                            p > 0.01          0.0 (0%)   
                 8                            p > 0.01      15.0 (71.0%)   
                 No Withdrawal                p < 0.01     21.0 (100.0%)   
                 Not Overdose                 p < 0.01     21.0 (100.0%)   
                 Overdose                     p < 0.01          0.0 (0%)   
                 Withdrawal                   p > 0.01          0.0 (0%)   
Comorbidity      alcohol-related disorders    p > 0.01        1.0 (5.0%)   
                 aspirin                      p > 0.01        1.0 (5.0%)   
                 diabetes mellitus            p > 0.01          0.0 (0%)   
                 hyperlipidemia               p > 0.01       4.0 (19.0%)   
                 ischemic heart disease       p > 0.01       2.0 (10.0%)   
                 mood disorders               p > 0.01       6.0 (29.0%)   
                 nicotine dependence          p > 0.01      16.0 (76.0%)   
                 obesity                      p > 0.01        1.0 (5.0%)   
                 systemic hypertension        p > 0.01      11.0 (52.0%)   
Demographic      African American or Hispanic p > 0.01      15.0 (71.0%)   
                 Female                       p > 0.01       9.0 (43.0%)   
                 Married                      p > 0.01       4.0 (19.0%)   
                 Unmarried                    p < 0.01      17.0 (81.0%)   
                 White                        p > 0.01       6.0 (29.0%)   
                 age                          p > 0.01      45.71 ± 5.62   
                 median_zip_income            p > 0.01       1.95 ± 0.51   
Insurance Status Medicaid                     p > 0.01      15.0 (71.0%)   
                 Medicare                     p < 0.01       2.0 (10.0%)   
                 Private Insurance            p > 0.01       3.0 (14.0%)   
                 Uninsured                    p < 0.01        1.0 (5.0%)   
Outcome          Cost                         p > 0.01   968.76 ± 563.74   
                 Died                         p < 0.01          0.0 (0%)   
                 ED Readmissions              p < 0.01      13.0 (62.0%)   
                 Inpatient Readmissions       p > 0.01        1.0 (5.0%)   

                                                       - Suboxone (N=15904)  
category         level_0                      ANOVA P                        
Clinical Pathway + Suboxone                   p > 0.01             0.0 (0%)  
                 - Suboxone                   p > 0.01     15904.0 (100.0%)  
                 0                            p > 0.01     15904.0 (100.0%)  
                 12                           p > 0.01             0.0 (0%)  
                 2                            p > 0.01             0.0 (0%)  
                 4                            p > 0.01             0.0 (0%)  
                 8                            p > 0.01             0.0 (0%)  
                 No Withdrawal                p < 0.01     15904.0 (100.0%)  
                 Not Overdose                 p < 0.01     15904.0 (100.0%)  
                 Overdose                     p < 0.01             0.0 (0%)  
                 Withdrawal                   p > 0.01             0.0 (0%)  
Comorbidity      alcohol-related disorders    p > 0.01       2856.0 (18.0%)  
                 aspirin                      p > 0.01         726.0 (5.0%)  
                 diabetes mellitus            p > 0.01       1909.0 (12.

In [6]:
create_summary(de_col_keys[0], "Complicated == False").to_excel(f"../tables/Table 1 {de_col_keys[0]} Uncomplicated.xlsx")
create_summary(de_col_keys[0], "Complicated == False")

C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Admitted (N=505)  \
category         level_0                      ANOVA P                      
Clinical Pathway Admitted                     p < 0.01    505.0 (100.0%)   
                 Complicated                  p > 0.01          0.0 (0%)   
                 Delayed                      p < 0.01      51.0 (10.0%)   
                 Did Not Obtain Surgery       p < 0.01     335.0 (66.0%)   
                 Immediate                    p < 0.01     119.0 (24.0%)   
                 New Cholecystitis            p > 0.01          0.0 (0%)   
                 No New Cholecystitis         p > 0.01    505.0 (100.0%)   
                 No Surgery                   p < 0.01     335.0 (66.0%)   
                 Not Admitted                 p < 0.01          0.0 (0%)   
                 Obtained Surgery             p < 0.01     170.0 (34.0%)   
                 Return Emergency             p > 0.01          0.0 (0%)   
                 Uncomplicated                p > 0.01    505.0 (100.0%)   
Comorbidity      alcohol-related disorders    p < 0.01       18.0 (4.0%)   
                 aspirin                      p > 0.01      64.0 (13.0%)   
                 diabetes mellitus            p < 0.01     121.0 (24.0%)   
                 hyperlipidemia               p < 0.01     178.0 (35.0%)   
                 ischemic heart disease       p < 0.01      89.0 (18.0%)   
                 mood disorders               p < 0.01     109.0 (22.0%)   
                 nicotine dependence          p < 0.01     186.0 (37.0%)   
                 obesity                      p < 0.01      98.0 (19.0%)   
                 systemic hypertension        p < 0.01     289.0 (57.0%)   
Demographic      African American or Hispanic p > 0.01     192.0 (38.0%)   
                 Female                       p < 0.01     329.0 (65.0%)   
                 Married                      p > 0.01     195.0 (39.0%)   
                 Unmarried                    p > 0.01     310.0 (61.0%)   
                 White                        p < 0.01     282.0 (56.0%)   
                 age                          p < 0.01      58.54 ± 1.85   
                 median_zip_income            p < 0.01       3.12 ± 0.09   
Insurance Status Medicaid                     p > 0.01     123.0 (24.0%)   
                 Medicare                     p < 0.01     224.0 (44.0%)   
                 Private Insurance            p < 0.01     137.0 (27.0%)   
                 Uninsured                    p < 0.01       14.0 (3.0%)   
Outcome          Cost                         p < 0.01  9548.31 ± 561.61   
                 Died                         p > 0.01          0.0 (0%)   
                 ED Readmissions              p > 0.01        7.0 (1.0%)   
                 Inpatient Readmissions       p > 0.01       14.0 (3.0%)   

                                                       Not Admitted (N=285)  
category         level_0                      ANOVA P                        
Clinical Pathway Admitted                     p < 0.01             0.0 (0%)  
                 Complicated                  p > 0.01             0.0 (0%)  
                 Delayed                      p < 0.01         57.0 (20.0%)  
                 Did Not Obtain Surgery       p < 0.01        228.0 (80.0%)  
                 Immediate                    p < 0.01             0.0 (0%)  
                 New Cholecystitis            p > 0.01             0.0 (0%)  
                 No New Cholecystitis         p > 0.01       285.0 (100.0%)  
                 No Surgery                   p < 0.01        228.0 (80.0%)  
                 Not Admitted                 p < 0.01       285.0 (100.0%)  
                 Obtained Surgery             p < 0.01         57.0 (20.0%)  
                 Return Emergency             p > 0.01             0.0 (0%)  
                 Uncomplicated                p > 0.01       285.0 (100.0%)  
Comorbidity      alcohol-related disorders    p < 0.01           1.0 (0.0%)

In [7]:
create_summary(de_col_keys[2], "Complicated == True").to_excel(f"../tables/Table 1 {de_col_keys[0]} Complicated.xlsx")
create_summary(de_col_keys[2], "Complicated == True")

C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Delayed (N=205)  \
category         level_0                      ANOVA P                        
Clinical Pathway Admitted                     p < 0.01       171.0 (83.0%)   
                 Complicated                  p > 0.01      205.0 (100.0%)   
                 Delayed                      p < 0.01      205.0 (100.0%)   
                 Did Not Obtain Surgery       p < 0.01            0.0 (0%)   
                 Immediate                    p < 0.01            0.0 (0%)   
                 New Cholecystitis            p > 0.01            0.0 (0%)   
                 No New Cholecystitis         p > 0.01      205.0 (100.0%)   
                 No Surgery                   p < 0.01            0.0 (0%)   
                 Not Admitted                 p < 0.01        34.0 (17.0%)   
                 Obtained Surgery             p < 0.01      205.0 (100.0%)   
                 Return Emergency             p < 0.01            0.0 (0%)   
                 Uncomplicated                p > 0.01            0.0 (0%)   
Comorbidity      alcohol-related disorders    p < 0.01          5.0 (2.0%)   
                 aspirin                      p < 0.01        31.0 (15.0%)   
                 diabetes mellitus            p < 0.01        58.0 (28.0%)   
                 hyperlipidemia               p < 0.01        81.0 (40.0%)   
                 ischemic heart disease       p < 0.01        38.0 (19.0%)   
                 mood disorders               p > 0.01        48.0 (23.0%)   
                 nicotine dependence          p < 0.01        76.0 (37.0%)   
                 obesity                      p < 0.01        49.0 (24.0%)   
                 systemic hypertension        p < 0.01       116.0 (57.0%)   
Demographic      African American or Hispanic p < 0.01        56.0 (27.0%)   
                 Female                       p > 0.01       122.0 (60.0%)   
                 Married                      p < 0.01       100.0 (49.0%)   
                 Unmarried                    p < 0.01       105.0 (51.0%)   
                 White                        p < 0.01       139.0 (68.0%)   
                 age                          p < 0.01         58.15 ± 2.4   
                 median_zip_income            p > 0.01          3.1 ± 0.15   
Insurance Status Medicaid                     p < 0.01        29.0 (14.0%)   
                 Medicare                     p < 0.01        83.0 (40.0%)   
                 Private Insurance            p < 0.01        85.0 (41.0%)   
                 Uninsured                    p > 0.01          5.0 (2.0%)   
Outcome          Cost                         p < 0.01  10146.73 ± 1059.02   
                 Died                         p > 0.01            0.0 (0%)   
                 ED Readmissions              p < 0.01          8.0 (4.0%)   
                 Inpatient Readmissions       p < 0.01        58.0 (28.0%)   

                                                       Immediate (N=3607)  \
category         level_0                      ANOVA P                       
Clinical Pathway Admitted                     p < 0.01    3607.0 (100.0%)   
                 Complicated                  p > 0.01    3607.0 (100.0%)   
                 Delayed                      p < 0.01           0.0 (0%)   
                 Did Not Obtain Surgery       p < 0.01           0.0 (0%)   
                 Immediate                    p < 0.01    3607.0 (100.0%)   
                 New Cholecystitis            p > 0.01           0.0 (0%)   
                 No New Cholecystitis         p > 0.01    3607.0 (100.0%)   
                 No Surgery                   p < 0.01           0.0 (0%)   
                 Not Admitted                 p < 0.01           0.0 (0%)   
                 Obtained Surgery             p < 0.01    3607.0 (100.0%)   
                 Return Emergency             p < 0.01           0.0 (0%)   
                 Uncomplicated                p > 0.01           0.0 (0%)   
Comorbidity     

In [8]:
create_summary(de_col_keys[1]).to_excel(f"../tables/Table 1 {de_col_keys[1]}.xlsx")
create_summary(de_col_keys[1])

C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Complicated (N=4882)  \
category         level_0                      ANOVA P                         
Clinical Pathway Admitted                     p < 0.01       4692.0 (96.0%)   
                 Complicated                  p < 0.01      4882.0 (100.0%)   
                 Delayed                      p < 0.01         205.0 (4.0%)   
                 Did Not Obtain Surgery       p < 0.01       1014.0 (21.0%)   
                 Immediate                    p < 0.01       3607.0 (74.0%)   
                 New Cholecystitis            p > 0.01             0.0 (0%)   
                 No New Cholecystitis         p > 0.01      4882.0 (100.0%)   
                 No Surgery                   p < 0.01       1014.0 (21.0%)   
                 Not Admitted                 p < 0.01         190.0 (4.0%)   
                 Obtained Surgery             p < 0.01       3868.0 (79.0%)   
                 Return Emergency             p < 0.01          56.0 (1.0%)   
                 Uncomplicated                p < 0.01             0.0 (0%)   
Comorbidity      alcohol-related disorders    p > 0.01         131.0 (3.0%)   
                 aspirin                      p > 0.01        488.0 (10.0%)   
                 diabetes mellitus            p > 0.01        932.0 (19.0%)   
                 hyperlipidemia               p > 0.01       1405.0 (29.0%)   
                 ischemic heart disease       p > 0.01        603.0 (12.0%)   
                 mood disorders               p > 0.01        911.0 (19.0%)   
                 nicotine dependence          p > 0.01       1588.0 (33.0%)   
                 obesity                      p < 0.01       1288.0 (26.0%)   
                 systemic hypertension        p > 0.01       2363.0 (48.0%)   
Demographic      African American or Hispanic p > 0.01       1782.0 (37.0%)   
                 Female                       p < 0.01       3058.0 (63.0%)   
                 Married                      p < 0.01       2324.0 (48.0%)   
                 Unmarried                    p < 0.01       2558.0 (52.0%)   
                 White                        p < 0.01       2761.0 (57.0%)   
                 age                          p > 0.01         53.72 ± 0.54   
                 median_zip_income            p > 0.01          3.18 ± 0.03   
Insurance Status Medicaid                     p > 0.01       1102.0 (23.0%)   
                 Medicare                     p > 0.01       1568.0 (32.0%)   
                 Private Insurance            p > 0.01       1882.0 (39.0%)   
                 Uninsured                    p > 0.01         243.0 (5.0%)   
Outcome          Cost                         p < 0.01    10301.77 ± 245.47   
                 Died                         p > 0.01           1.0 (0.0%)   
                 ED Readmissions              p > 0.01         154.0 (3.0%)   
                 Inpatient Readmissions       p < 0.01         215.0 (4.0%)   

                                                       Uncomplicated (N=790)  
category         level_0                      ANOVA P                         
Clinical Pathway Admitted                     p < 0.01         505.0 (64.0%)  
                 Complicated                  p < 0.01              0.0 (0%)  
                 Delayed                      p < 0.01         108.0 (14.0%)  
                 Did Not Obtain Surgery       p < 0.01         563.0 (71.0%)  
                 Immediate                    p < 0.01         119.0 (15.0%)  
                 New Cholecystitis            p > 0.01              0.0 (0%)  
                 No New Cholecystitis         p > 0.01        790.0 (100.0%)  
                 No Surgery                   p < 0.01         563.0 (71.0%)  
                 Not Admitted                 p < 0.01         285.0 (36.0%)  
                 Obtained Surgery             p < 0.01         227.0 (29.0%)  
                 Return Emergency             p < 0.01              0.0 (0%)  
                 Uncompli

In [9]:
create_summary(de_col_keys[2]).to_excel(f"../tables/Table 1 {de_col_keys[2]}.xlsx")
create_summary(de_col_keys[2])

C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\Michael\anaconda3\envs\hcup-study-figs\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Delayed (N=313)  \
category         level_0                      ANOVA P                     
Clinical Pathway Admitted                     p < 0.01    222.0 (71.0%)   
                 Complicated                  p < 0.01    205.0 (65.0%)   
                 Delayed                      p < 0.01   313.0 (100.0%)   
                 Did Not Obtain Surgery       p < 0.01         0.0 (0%)   
                 Immediate                    p < 0.01         0.0 (0%)   
                 New Cholecystitis            p > 0.01         0.0 (0%)   
                 No New Cholecystitis         p > 0.01   313.0 (100.0%)   
                 No Surgery                   p < 0.01         0.0 (0%)   
                 Not Admitted                 p < 0.01     91.0 (29.0%)   
                 Obtained Surgery             p < 0.01   313.0 (100.0%)   
                 Return Emergency             p < 0.01         0.0 (0%)   
                 Uncomplicated                p < 0.01    108.0 (35.0%)   
Comorbidity      alcohol-related disorders    p < 0.01       6.0 (2.0%)   
                 aspirin                      p < 0.01     36.0 (12.0%)   
                 diabetes mellitus            p < 0.01     78.0 (25.0%)   
                 hyperlipidemia               p < 0.01    108.0 (35.0%)   
                 ischemic heart disease       p < 0.01     47.0 (15.0%)   
                 mood disorders               p > 0.01     62.0 (20.0%)   
                 nicotine dependence          p < 0.01    105.0 (34.0%)   
                 obesity                      p < 0.01     67.0 (21.0%)   
                 systemic hypertension        p < 0.01    160.0 (51.0%)   
Demographic      African American or Hispanic p < 0.01     90.0 (29.0%)   
                 Female                       p > 0.01    205.0 (65.0%)   
                 Married                      p < 0.01    150.0 (48.0%)   
                 Unmarried                    p < 0.01    163.0 (52.0%)   
                 White                        p < 0.01    204.0 (65.0%)   
                 age                          p < 0.01     54.41 ± 2.04   
                 median_zip_income            p > 0.01      3.19 ± 0.11   
Insurance Status Medicaid                     p < 0.01     51.0 (16.0%)   
                 Medicare                     p < 0.01    104.0 (33.0%)   
                 Private Insurance            p < 0.01    147.0 (47.0%)   
                 Uninsured                    p > 0.01       7.0 (2.0%)   
Outcome          Cost                         p < 0.01  8601.6 ± 840.09   
                 Died                         p > 0.01         0.0 (0%)   
                 ED Readmissions              p < 0.01      14.0 (4.0%)   
                 Inpatient Readmissions       p < 0.01     64.0 (20.0%)   

                                                       Immediate (N=3726)  \
category         level_0                      ANOVA P                       
Clinical Pathway Admitted                     p < 0.01    3726.0 (100.0%)   
                 Complicated                  p < 0.01     3607.0 (97.0%)   
                 Delayed                      p < 0.01           0.0 (0%)   
                 Did Not Obtain Surgery       p < 0.01           0.0 (0%)   
                 Immediate                    p < 0.01    3726.0 (100.0%)   
                 New Cholecystitis            p > 0.01           0.0 (0%)   
                 No New Cholecystitis         p > 0.01    3726.0 (100.0%)   
                 No Surgery                   p < 0.01           0.0 (0%)   
                 Not Admitted                 p < 0.01           0.0 (0%)   
                 Obtained Surgery             p < 0.01    3726.0 (100.0%)   
                 Return Emergency             p < 0.01           0.0 (0%)   
                 Uncomplicated                p < 0.01       119.0 (3.0%)   
Comorbidity      alcohol-related disorders    p < 0.01        68.0 (2.0%)   
                 aspirin                      p < 